In [77]:
import requests
import pandas as pd
import numpy as np
import json
from glob import glob
#import streamlit as st
#from st_aggrid import AgGrid, GridOptionsBuilder, GridUpdateMode
from io import BytesIO
import base64
import subprocess
import os
from collections import Counter
import numpy as np
import wget
from tqdm import tqdm
#from ST_varvisDownloader import varvis_api_login, get_analysisIds_per_person, get_download_link, collect_download_links
import pysam
from time import time
from multiprocessing import Pool,Manager

In [115]:
from tqdm.contrib.concurrent import process_map

/home/drukewitz/miniconda3_new/envs/Jupyter/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def varvis_api_login(sessionId, token, target, user_name, password):
    
    # 1) Get CSRF token and session ID to log in
    r = requests.head("https://" + target + ".varvis.com/authenticate")
    token = r.headers['X-CSRF-TOKEN']
    sessionId = r.cookies['session']
     
    # 2) Log in and update CSRF token in an additional step
    r = requests.post("https://" + target + ".varvis.com/login", data = {'_csrf': token, 'username': user_name, 'password': password}, cookies = dict(session=sessionId))
    sessionId = r.cookies['session']
    
    ### THIS ADDITIONAL REQUEST IS NECESSARY TO RETRIEVE THE CORRECT CSRF TOKEN:
    r = requests.head("https://" + target + ".varvis.com/authenticate", cookies = dict(session=sessionId))
    token = r.headers['X-CSRF-TOKEN']
    
    return sessionId, token

def get_analysisIds_per_person(personId, sessionId):
    
    r = requests.get("https://" + target + ".varvis.com/person/" + personId + "/analyses", cookies = dict(session=sessionId))

    return json.loads(r.text)

def get_download_link(analysisId, sessionId):
    
    r = requests.get("https://" + target + ".varvis.com/api/analysis/" + analysisId + "/get-file-download-links", cookies = dict(session=sessionId))
    
    response = json.loads(r.text)
    
    return response


def collect_download_links(personIds, sessionId):
    analysisIds = []
    limsIds= []
    sampleIds= []
    all_fileNames= []
    all_links= []
    fileNames = []
    AnType=[]
    links = []
    for person in personIds:
        print(person)
        analysis_response = get_analysisIds_per_person(person, sessionId)
        if not type(analysis_response) == list:
            if "errorMessageId" in analysis_response.keys():
                AnType.append('personID not found')
                analysisIds.append("personID not found")
                limsIds.append(person)
                sampleIds.append("personID not found")
                fileNames.append("personID not found")
                links.append("personID not found")
                
                continue # this skips the rest of the loop and continues with the next personID
            
        download_responses = []
        # !!!! Download links can only be obtained for 'SNV' analyses
        
        for row in analysis_response:
            analysisId =str(row["id"])
            download_response = get_download_link(analysisId, sessionId)
            
            if download_response['errorExpected']:
                continue
            apiFileLinks = download_response["response"]["apiFileLinks"]
            for file in apiFileLinks:
                AnType.append(row['enrichmentKitName'])
                analysisIds.append(download_response["response"]["id"])
                limsIds.append(download_response["response"]["limsId"])
                sampleIds.append(download_response["response"]["sampleId"])
                fileNames.append(file["fileName"])
                links.append(file["downloadLink"])
    df_overview = pd.DataFrame()  
    df_overview["analysisId"] = analysisIds
    df_overview['analysisType']=AnType
    df_overview["limsIds"] = limsIds
    df_overview["sampleIds"] = sampleIds
    df_overview["fileNames"] = fileNames
    df_overview["links"] = links

    return df_overview

In [3]:
target='uni-leipzig'
user_name='hugapi'
password=""
sessionId = ""
token = ""


In [ ]:
with open('../../00_fullBams/Repeat/data/variant_catalog_with_offtargets.GRCh37.json') as j:
    x=j.read()
aa =json.loads(x)
positions = set()
chroms=[]
starts=[]
ends=[]

for roi in aa:
    RR = roi['ReferenceRegion']
    if type(RR)==str:
        positions.add(RR)
    else:
        for o in RR:
            positions.add(o)
    if 'OfftargetRegions' not in roi:
        continue
    OR = roi['OfftargetRegions']
    for o in OR:
        positions.add(o)

        
chroms = [x.split(':')[0].replace('chr','') for x in positions]
starts = [int(x.split(':')[1].split('-')[0]) for x in positions]
ends = [int(x.split(':')[1].split('-')[1]) for x in positions]

In [159]:
def GetChunk(x):
    bam = x[0]
    bai = x[1]
    c = x[2]
    s = x[3]
    e = x[4]
    padding = x[5]
    reads = set()
    with pysam.AlignmentFile(bam,filepath_index=bai) as f:
        for r in f.fetch(c,s-padding,e+padding,until_eof=True):
            r=r.to_string()
            if r not in reads:
                reads.add(r)
    return reads

def GetBam(bam,bai,chrom,start,stop,outname,padding=50,proc=12):
    mp_split = [(bam,bai,c,s,e,padding) for c,s,e in zip(chrom,start,stop)]
    with Pool(processes=proc) as p:
       # r = p.map(GetChunk,tqdm(mp_split,total=len(mp_split)))
        r = list(tqdm(p.imap(GetChunk, mp_split), total=len(mp_split)))
    rr = [xx for x in r for xx in x]    
    outname_temp = outname.replace('.bam','unsorted.bam')
        
    with pysam.AlignmentFile(bam,filepath_index=bai) as f:
        with pysam.AlignmentFile(outname_temp,'wb',template=f) as bamout:
            for r in rr:
                bamout.write(pysam.AlignedSegment.fromstring(r,f.header))
                
    pysam.sort("-o", outname, outname_temp)
    pysam.index(outname)


In [164]:
with open('../../00_fullBams/Repeat/bams.txt') as b:
    sessionId, token = varvis_api_login(sessionId, token, target, user_name, password)
    for bam in b:
        x=time()
        i = bam.split('/')[-1].split('_')[0]
        print(i)
        d = collect_download_links([i],sessionId)
        d_bam = d[(d['analysisType'].str.contains('Exome')) & (d['fileNames'].str.endswith('.bam'))]
        d_bai = d[(d['analysisType'].str.contains('Exome')) & (d['fileNames'].str.endswith('.bai'))]
        for n,(bam,bai) in enumerate(zip(d_bam['links'],d_bai['links'])):
            with pysam.AlignmentFile(bam,filepath_index=bai) as f:
                pass
            GetBam(bam,bai,chroms,starts,ends,f'{n}_{i}_varvis.bam',padding=600, proc=100)
        print(time()-x)
        

2029421
2029421


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:02<00:00, 60.31it/s]


7.506390333175659
2029546
2029546


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:03<00:00, 44.80it/s]


8.61121916770935
2030152
2030152


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:03<00:00, 45.66it/s]


8.784922122955322
2030181
2030181


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:03<00:00, 44.28it/s]


9.048022985458374
